# Gamut Mapping - Ramblings 02

In [1]:
%matplotlib widget

In [2]:
from __future__ import division, unicode_literals

import colour
import matplotlib.pyplot as plt
import numpy as np
import os
import scipy.optimize
from functools import reduce

COLOUR_STYLE = colour.plotting.colour_style()
COLOUR_STYLE.update({
    'figure.figsize': (10.24, 10.24),
    'legend.framealpha':
    colour.plotting.COLOUR_STYLE_CONSTANTS.opacity.low
})

plt.style.use(COLOUR_STYLE)

plt.style.use('dark_background')

colour.utilities.describe_environment()

colour.utilities.filter_warnings(*[True] * 4);

*                                                                             *
*   Interpreter :                                                             *
*       python : 3.7.6 (default, Dec 30 2019, 19:38:26)                       *
*                [Clang 11.0.0 (clang-1100.0.33.16)]                          *
*                                                                             *
*   colour-science.org :                                                      *
*       colour : v0.3.15-141-g3bebd7e9                                        *
*                                                                             *
*   Runtime :                                                                 *
*       imageio : 2.8.0                                                       *
*       matplotlib : 3.0.3                                                    *
*       numpy : 1.18.4                                                        *
*       scipy : 1.4.1                   

## Colour Stripe Generation

In [3]:
def colour_stripe(samples=360):
    H = np.linspace(0, 1, samples)

    HSV = colour.utilities.tstack([H, np.ones(samples) * 0.1, np.ones(samples)])
    RGB = colour.HSV_to_RGB(HSV)
 
    return RGB[np.newaxis, ...]

In [4]:
COLOUR_STRIPE = colour_stripe();

colour.plotting.plot_image(np.resize(COLOUR_STRIPE, [360, 360, 3]));

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

## Compression Function

In [5]:
def tanh_compression_function(x, a=0.8, b=1 - 0.8):
    x = colour.utilities.as_float_array(x)

    return np.where(x > a, (a + b * np.tanh((x - a) / b)), x)

## Comparison

In [6]:
SAMPLES = np.linspace(0, 1, 360)


def sine_wave(x, a, b, c):
    return np.sin(x * a + b) * c


def medicina_HSV_control(p=(np.pi * 6, np.pi, np.pi / 180), RGB=COLOUR_STRIPE, S_c=0, C_f=tanh_compression_function):    
    H, S, V = colour.utilities.tsplit(colour.RGB_to_HSV(RGB))

    HSV = colour.utilities.tstack(
        [(H + sine_wave(SAMPLES, *p))  % 1, C_f(S, S_c, 1 - S_c), V])

    return colour.HSV_to_RGB(HSV)


def medicina_RGB_saturation(RGB=COLOUR_STRIPE, C_t=1, C_f=tanh_compression_function):
    C_t = 1 - C_t

    L = np.max(RGB, axis=-1)[..., np.newaxis]

    D = np.abs(RGB - L) / L

    D_c = C_f(D, C_t, 1 - C_t)

    RGB_c = L - D_c * L

    return RGB_c

RGB_S = medicina_RGB_saturation()

In [7]:
colour.plotting.plot_image(np.resize(medicina_HSV_control(), [360, 360, 3]));

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [8]:
colour.plotting.plot_image(np.resize(RGB_S, [360, 360, 3]));

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [9]:
colour.plotting.plot_image(np.resize(np.abs(medicina_HSV_control([0, 0, 0]) - RGB_S) * 5, [360, 360, 3]));

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

## Optimisation

After some initial tests, it seemed like the closest function to minimize the error was a triangle or sine wave: https://community.acescentral.com/t/gamut-mapping-in-cylindrical-and-conic-spaces/2870/10?u=thomas_mansencal

$\sin\bigg(x \cdot \pi \cdot 6 + \pi\bigg) * \cfrac{\pi}{\approx180}$

is almost the best fitting function for this case, the rounded 180 just happens to be convenient here but the proper value is a bit lower.

In [10]:
def objective_function(abc):
    return np.sum(np.abs(medicina_HSV_control(abc) - RGB_S))


I = 1
def callback(*args):
    global I
    print(I, args[0][:10])
    I += 1

    
P = scipy.optimize.minimize(objective_function, (np.pi * 6, np.pi, np.pi / 180), callback=callback)

1 [  1.88495706e+01   3.14159002e+00   2.33152982e-04]
2 [  1.88494518e+01   3.14161035e+00   2.16193127e-04]
3 [  1.88473238e+01   3.14161992e+00   2.04776638e-04]
4 [  1.88458403e+01   3.14161467e+00   2.08286054e-04]
5 [  1.88335846e+01   3.14159124e+00   2.08841520e-04]
6 [  1.88089644e+01   3.14320272e+00   2.08851152e-04]
7 [  1.87889600e+01   3.14710192e+00   2.08644792e-04]
8 [  1.87506788e+01   3.15676733e+00   2.08857361e-04]
9 [  1.87330109e+01   3.16606204e+00   2.09020782e-04]
10 [  1.87138011e+01   3.18097114e+00   2.09232951e-04]
11 [  1.86803937e+01   3.21214832e+00   2.09074508e-04]
12 [  1.86553859e+01   3.23652649e+00   2.09048112e-04]
13 [  1.86593343e+01   3.23547217e+00   2.09196037e-04]
14 [  1.86603827e+01   3.23486540e+00   2.09256701e-04]
15 [  1.86611743e+01   3.23440224e+00   2.09279354e-04]
16 [  1.86624630e+01   3.23363892e+00   2.09278507e-04]
17 [  1.86626527e+01   3.23386962e+00   2.09257098e-04]
18 [  1.86636585e+01   3.23364845e+00   2.09248875e-04]
1

In [11]:
P

      fun: 0.0034695445150292326
 hess_inv: array([[  1.31925509e+01,  -2.06802781e-01,  -4.99047609e-05],
       [ -2.06802781e-01,   2.51683377e-01,  -6.01824562e-06],
       [ -4.99047609e-05,  -6.01824562e-06,   9.81249095e-09]])
      jac: array([  9.70065594e-06,   4.56795096e-05,  -4.83787395e-01])
  message: 'Desired error not necessarily achieved due to precision loss.'
     nfev: 902
      nit: 20
     njev: 178
   status: 2
  success: False
        x: array([  1.86653666e+01,   3.23371579e+00,   2.09242010e-04])

In [12]:
colour.plotting.artist()
plt.plot(SAMPLES, np.sin(np.linspace(0, 1, len(SAMPLES)) * P.x[0] + P.x[1]) * P.x[2])
for i in np.arange(0, 360 + 60, 60) / 360:
    plt.axvline(i, c='r')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [13]:
print(np.max(np.abs(medicina_HSV_control() - RGB_S)))
print(np.max(np.abs(medicina_HSV_control(P.x) - RGB_S)))


colour.plotting.plot_image(np.resize(np.abs(medicina_HSV_control(P.x) - RGB_S) * 5, [360, 360, 3]));

0.0103128461949
2.57702575792e-05


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …